In [1]:
# 1. load data
import sys
import os
from dotenv import load_dotenv
sys.path.append(os.path.abspath("../src"))

from data_loader import load_and_prepare_projects, load_prediction_set

path_22 = '../data/projects2022.csv'
path_24 = '../data/projects2024.csv'

df, df22_shuffled = load_and_prepare_projects(path_22,path_24)

ids_to_predict_path = '../data/projects_to_predict_2024.csv'
test = load_prediction_set(df, ids_to_predict_path)

In [2]:
df

,project_id,project_name,description,category,cost,district,votes,district_number,rank,year
0,136,Piste cyclable avenue Saint-Exupéry,"Lorsque l'on habite vers la place de l'ormeau,...",Éco-mobilité,200000.0,12 - Pont des Demoiselles / Ormeau / Montaudra...,492,12,1,2022
1,7,Faire des voies vélos et piétons dédiées et bi...,Pour favoriser la cohabitation piétons/cyclist...,Éco-mobilité,150000.0,1 - Capitole / Arnaud Bernard / Carmes,467,1,2,2022
2,5,Végétalisation de la place du Capitole,Pour donner encore plus de majesté à ce lieu e...,Nature en ville,200000.0,1 - Capitole / Arnaud Bernard / Carmes,358,1,3,2022
3,132,Lutte anti-moustiques,Mise en place de pièges à moustiques dans le q...,Nature en ville,7500.0,12 - Pont des Demoiselles / Ormeau / Montaudra...,258,12,4,2022
4,71,Planter des arbres et jardins partagés partout...,Une voisine exploite un bout du terrain du sup...,Nature en ville,20000.0,8 - Minimes / Barrière de Paris / Ponts-Jumeau...,243,8,5,2022
...,...,...,...,...,...,...,...,...,...,...
378,372,Installation d'un portique pour réparer les vé...,Fleury : Place André Abbal (Reynerie): Il sera...,Éco-mobilité et transports,5000.0,17 - Mirail-Université / Reynerie / Bellefontaine,20,17,179,2024
379,356,Débitumiser les places de stationnement rue Be...,Je propose d’enlever le bitume sur les places ...,Nature en ville et biodiversité,240000.0,15 - Croix de Pierre / Route d'Espagne,19,15,180,2024
380,230,Installation de poubelles à déjection aux Sept...,"Installer sur le quartier des sept deniers, de...",Cadre de vie,6000.0,7 - Sept Deniers / Ginestous-Sesquières / Lalande,19,7,181,2024
381,370,Installer des tables et des chaises au Parc Ma...,Il n’y a pas assez de tables et de chaises pou...,Cadre de vie,85000.0,17 - Mirail-Université / Reynerie / Bellefontaine,18,17,182,2024


#### Prompt 1: Cost Estimation GPT-4-Turbo

In [3]:
from prompt_utils import tokens_counter, prompt_cost
from prompt_builder import build_prompt, get_all_projects_from_22_election

last_projects_results = get_all_projects_from_22_election(df22_shuffled)

test['prompt'] = test.apply(
    lambda x: build_prompt(
        '../prompts/prompt_template1.txt',
        {
            'last_projects_results': last_projects_results,
            'project_name': x['project_name'],
            'cost': x['cost'],
            'district': x['district'],
            'description': x['description']
        }
    ),
    axis=1
)

test['n_tokens'] = test['prompt'].apply(lambda p: tokens_counter(p))
test['cost_usd'] = test['n_tokens'].apply(lambda n_tokens: prompt_cost(n_tokens, 'gpt-4-turbo'))

print('mean tokens by prompt: {:.2f}'.format(test.n_tokens.mean()))
print('experiment total cost: {:.2f}'.format(test.cost_usd.sum()))

mean tokens by prompt: 13454.16
experiment total cost: 4.98


#### Prompt 2 (CoT): Cost Estimation GPT-4-Turbo

In [4]:
from prompt_utils import tokens_counter, prompt_cost
from prompt_builder import build_prompt, get_all_projects_from_22_election

last_projects_results = get_all_projects_from_22_election(df22_shuffled)

test['prompt'] = test.apply(
    lambda x: build_prompt(
        '../prompts/prompt_template3_CoT.txt',
        {
            'last_projects_results': last_projects_results,
            'project_name': x['project_name'],
            'cost': x['cost'],
            'district': x['district'],
            'description': x['description']
        }
    ),
    axis=1
)

test['n_tokens'] = test['prompt'].apply(lambda p: tokens_counter(p))
test['cost_usd'] = test['n_tokens'].apply(lambda n_tokens: prompt_cost(n_tokens, 'gpt-4-turbo'))

print('mean tokens by prompt: {:.2f}'.format(test.n_tokens.mean()))
print('avg.cost of each prediction: ${:.2f}'.format(test.cost_usd.mean()))
print('experiment total cost: {:.2f}'.format(test.cost_usd.sum()))


mean tokens by prompt: 13547.16
avg.cost of each prediction: $0.14
experiment total cost: 5.01


#### Prompt 4: (RAG + CoT): Cost Estimation GPT-4-Turbo

In [7]:
import prompt_builder as pb

conn_params = {
    "host": os.environ["PG_HOST"],
    "database": os.environ["PG_DATABASE"],
    "user": os.environ["PG_USER"],
    "password": os.environ['PG_PASSWORD']
}


print(pb.get_top_k_similar_projects_in_22_by_district(df,136,conn_params, k=5))


- Restructuration "verte" de la place Roger Arnaud: (Coût: 200000.0 €, District: 12 - Pont des Demoiselles / Ormeau / Montaudran / La Terrasse / Malepère). 112 Voix, Classement: 26
- Installation d’un parcours de santé au quartier Montaudran vers la Halle de la Machine: (Coût: 150000.0 €, District: 12 - Pont des Demoiselles / Ormeau / Montaudran / La Terrasse / Malepère). 76 Voix, Classement: 49
- Création d une piste cyclable avenue de la Marcaissonne: (Coût: 200000.0 €, District: 12 - Pont des Demoiselles / Ormeau / Montaudran / La Terrasse / Malepère). 52 Voix, Classement: 71
- Le développement de l'auto partage: (Coût: 4500.0 €, District: 12 - Pont des Demoiselles / Ormeau / Montaudran / La Terrasse / Malepère). 35 Voix, Classement: 108
- Aménagement parc rue du Canigou: (Coût: 150000.0 €, District: 12 - Pont des Demoiselles / Ormeau / Montaudran / La Terrasse / Malepère). 11 Voix, Classement: 164



In [8]:
df

,project_id,project_name,description,category,cost,district,votes,district_number,rank,year
0,136,Piste cyclable avenue Saint-Exupéry,"Lorsque l'on habite vers la place de l'ormeau,...",Éco-mobilité,200000.0,12 - Pont des Demoiselles / Ormeau / Montaudra...,492,12,1,2022
1,7,Faire des voies vélos et piétons dédiées et bi...,Pour favoriser la cohabitation piétons/cyclist...,Éco-mobilité,150000.0,1 - Capitole / Arnaud Bernard / Carmes,467,1,2,2022
2,5,Végétalisation de la place du Capitole,Pour donner encore plus de majesté à ce lieu e...,Nature en ville,200000.0,1 - Capitole / Arnaud Bernard / Carmes,358,1,3,2022
3,132,Lutte anti-moustiques,Mise en place de pièges à moustiques dans le q...,Nature en ville,7500.0,12 - Pont des Demoiselles / Ormeau / Montaudra...,258,12,4,2022
4,71,Planter des arbres et jardins partagés partout...,Une voisine exploite un bout du terrain du sup...,Nature en ville,20000.0,8 - Minimes / Barrière de Paris / Ponts-Jumeau...,243,8,5,2022
...,...,...,...,...,...,...,...,...,...,...
378,372,Installation d'un portique pour réparer les vé...,Fleury : Place André Abbal (Reynerie): Il sera...,Éco-mobilité et transports,5000.0,17 - Mirail-Université / Reynerie / Bellefontaine,20,17,179,2024
379,356,Débitumiser les places de stationnement rue Be...,Je propose d’enlever le bitume sur les places ...,Nature en ville et biodiversité,240000.0,15 - Croix de Pierre / Route d'Espagne,19,15,180,2024
380,230,Installation de poubelles à déjection aux Sept...,"Installer sur le quartier des sept deniers, de...",Cadre de vie,6000.0,7 - Sept Deniers / Ginestous-Sesquières / Lalande,19,7,181,2024
381,370,Installer des tables et des chaises au Parc Ma...,Il n’y a pas assez de tables et de chaises pou...,Cadre de vie,85000.0,17 - Mirail-Université / Reynerie / Bellefontaine,18,17,182,2024


In [13]:
import prompt_builder as pb
from prompt_utils import tokens_counter, prompt_cost

load_dotenv()

conn_params = {
    "host": os.environ["PG_HOST"],
    "database": os.environ["PG_DATABASE"],
    "user": os.environ["PG_USER"],
    "password": os.environ['PG_PASSWORD']
}

top_k_voted_22 = pb.get_top_k_projects_from_22_election(df, k=10)
proj_count_avg_by_district= pb.get_project_count_and_avg_votes_by_district(df)
proj_count_avg_by_categories = pb.get_project_count_and_avg_votes_by_categories(df)

test['prompt'] = test.apply(
    lambda x: pb.build_prompt(
        '../prompts/prompt_rag2_CoT.txt',
        {
            'top_k_voted_22': top_k_voted_22,
            'project_count_and_avg_votes_by_district': proj_count_avg_by_district,
            'project_count_and_avg_votes_by_categories': proj_count_avg_by_categories,
            'project_name': x['project_name'],
            'cost': x['cost'],
            'district': x['district'],
            'description': x['description'],
            'top_k_similar_projects_in_22': pb.get_top_k_similar_projects_in_22(df,x['project_id'],conn_params, k=10),
            'count_of_projects22_in_quartier': pb.get_count_of_projects_in_quartier(df,x['district']),
            'top_k_voted_in_district': pb.get_top_k_voted_in_district(df, x['district'], k=5),
            'count_of_projects24_in_quartier': pb.get_count_of_projects_in_quartier(df,x['district'], year=2024),
            'top_k_similar_projects_in_district': pb.get_top_k_similar_projects_in_22_by_district(df, x['project_id'], conn_params,k=5)
        }
    ),
    axis=1
)

test['n_tokens'] = test['prompt'].apply(lambda p: tokens_counter(p))
test['cost_usd'] = test['n_tokens'].apply(lambda n_tokens: prompt_cost(n_tokens, 'gpt-4-turbo'))

print('mean tokens by prompt: {:.2f}'.format(test.n_tokens.mean()))
print('avg.cost of each prediction: ${:.2f}'.format(test.cost_usd.mean()))
print('experiment total cost: {:.2f}'.format(test.cost_usd.sum()))

mean tokens by prompt: 3949.70
avg.cost of each prediction: $0.04
experiment total cost: 1.46


In [14]:
print(test['prompt'][0])

Vous êtes un modèle expert dans l’analyse des élections de budgets participatifs, spécifiquement dans le contexte de la ville de Toulouse, en France.

À Toulouse, la municipalité organise des élections de budget participatif pour financer des projets citoyens. Chaque habitant peut voter pour entre 1 et 3 initiatives sur le site web officiel. Les votes sont anonymes.

Voici comment les projets sont sélectionnés :
- Tous les projets sont classés selon le nombre de voix obtenues.
- Un algorithme « greedy » est utilisé : on finance les projets dans l’ordre décroissant des voix, tant que le budget total n’est pas dépassé.
- Si un projet est trop coûteux pour le budget restant, on passe au suivant.

---

Lors de l’élection de 2022 :
- Il y a eu **4532 votants**, chaque personne pouvant voter pour jusqu'à 3 projets.
- Au total, environ **11918 voix** ont été exprimées sur **200 projets**.
- Le nombre de voix par projet variait de *2* à 492, avec une moyenne autour de **58** voix et un écart-t

#### Calulo en Llama 3.1 70b

In [ ]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("meta-llama/Meta-Llama-3.1-8B-Instruct")
prompt = "Tu texto aquí"
n_tokens = len(tokenizer.encode(prompt))
print("Tokens:", n_tokens)